In [1]:
# Import the required libraries
import kaggle
import pandas as pd
from datetime import datetime
from preprocessing import add_future_records, create_date_range, add_time0_data


In [2]:
# Ensure the Kaggle environment is set up (you might need to manually import your Kaggle API key)
kaggle.api.authenticate()

# Kaggle Dataset URL components
dataset_url = "gsagar12/dspp1"
filename = "customer_cases.csv"

# Download dataset
# save to  one level up from current directory
kaggle.api.dataset_download_files(dataset_url, path='../Data', unzip=True)


Dataset URL: https://www.kaggle.com/datasets/gsagar12/dspp1


In [3]:
# Load the dataset into a DataFrames
purchase_df = pd.read_csv('../Data/customer_product.csv')
customers_info_df = pd.read_csv('../Data/customer_info.csv')
prices_df = pd.read_csv('../Data/product_info.csv')

print(f'Customers: {purchase_df.shape}, Products: {prices_df.shape}, Customer Info: {customers_info_df.shape}')


Customers: (508932, 5), Products: (2, 4), Customer Info: (508932, 4)


In [4]:
# merge data
merged_df = purchase_df.merge(prices_df, 
                        left_on='product', 
                        right_on='product_id',
                        how='left')
full_raw_data = merged_df.merge(customers_info_df,
              on='customer_id',
                how='left')

print(f'Full data: {full_raw_data.shape}')

Full data: (508932, 12)


In [5]:
full_raw_data['signup_date_time'] = pd.to_datetime(full_raw_data['signup_date_time'])
full_raw_data['cancel_date_time'] = pd.to_datetime(full_raw_data['cancel_date_time']).fillna(pd.to_datetime(datetime.now().date()))
full_raw_data['cohort'] = full_raw_data['signup_date_time'].apply(lambda x: x.replace(day=1).date())
full_raw_data['last_payment'] = full_raw_data['cancel_date_time'].apply(lambda x: x.replace(day=1).date())
full_raw_data['cohort'] = pd.to_datetime(full_raw_data['cohort'])
full_raw_data['last_payment'] = pd.to_datetime(full_raw_data['last_payment'])


full_raw_data.sort_values(by=['cohort', 'last_payment'], inplace=True)

In [6]:
# interpolate the data so that for each "User ID" there will be a row for a date between the "Join Date" and "Last Payment Date" date, one date per month
full_raw_data['current_date'] = full_raw_data.apply(lambda row: create_date_range(row['cohort'], row['last_payment']), axis=1)

full_processed_data = full_raw_data.explode('current_date')
full_processed_data = full_processed_data.drop(columns=['product_id', 'product', 'signup_date_time', 'cancel_date_time', 'age'])

print(f'Full data: {full_processed_data.shape}')

Full data: (24321317, 10)


In [7]:
full_processed_data['actual_payment'] = full_processed_data.apply(lambda row: row['price'] if row['name'] == 'monthly_subscription' else row['price'] if row['cohort'].month == row['current_date'].month else 0, axis=1)
# calculate the number of months since the user joined
full_processed_data['time_since_attribution'] = full_processed_data.apply(lambda row: (row.current_date.year - row.cohort.year) * 12 + (row.current_date.month - row.cohort.month), axis=1)
full_processed_data = full_processed_data.dropna(subset=['time_since_attribution'])
full_processed_data['time_since_attribution'] = full_processed_data['time_since_attribution'].astype(int)

print(f'Full data: {full_processed_data.shape}')

Full data: (24321317, 12)


In [8]:
# keep up to month 12
filtered_full_processed_data = full_processed_data[full_processed_data['time_since_attribution'] <= 12]

In [9]:
filtered_full_processed_data = filtered_full_processed_data.rename(columns={'name': 'subscription',
                        'actual_payment': 'revenue'}).drop(columns=['billing_cycle', 'price'])


In [10]:
# Step 1: Calculate unique customer_id counts for each subscription type within each group
monthly_counts = filtered_full_processed_data[filtered_full_processed_data['subscription'] == 'monthly_subscription'].groupby(
    ['gender', 'cohort', 'time_since_attribution', 'current_date']
)['customer_id'].nunique().reset_index(name='monthly_subs')

annual_counts = filtered_full_processed_data[filtered_full_processed_data['subscription'] == 'annual_subscription'].groupby(
    ['gender', 'cohort', 'time_since_attribution', 'current_date']
)['customer_id'].nunique().reset_index(name='annual_subs')

# Step 2: Aggregate revenue by the specified groups
revenue_agg = filtered_full_processed_data.groupby(['gender', 'cohort', 'time_since_attribution', 'current_date']).agg(
    revenue=('revenue', 'sum')
).reset_index()

# Step 3: Merge the unique subscriber counts with the aggregated revenue
agg_data_df = revenue_agg.merge(monthly_counts, on=['gender', 'cohort', 'time_since_attribution', 'current_date'], how='left')
agg_data_df = agg_data_df.merge(annual_counts, on=['gender', 'cohort', 'time_since_attribution', 'current_date'], how='left')

# Fill NaN values with 0s for counts in case there are groups without monthly or annual subscribers
agg_data_df.fillna({'monthly_subs': 0, 'annual_subs': 0}, inplace=True)



In [11]:
agg_data_df['revenue'] = agg_data_df['revenue'].astype(int)
adjusted_df = agg_data_df.sort_values(by=['gender', 'cohort', 'time_since_attribution'])


In [12]:
adjusted_df = add_future_records(adjusted_df.drop(columns=['current_date']), id_columns=['cohort', 'gender', 'time_since_attribution'])
adjusted_df["current_date"] = adjusted_df.apply(
        lambda x: x.cohort + pd.DateOffset(months=x.time_since_attribution), axis=1
    )
adjusted_df.sort_values(by=[ 'gender', 'cohort', 'time_since_attribution'], inplace=True)
adjusted_df['cumulative_revenue'] = adjusted_df.groupby(['gender', 'cohort'])['revenue'].cumsum()
adjusted_df.shape

(1560, 8)

In [21]:
# add time0 revenue
adjusted_df = add_time0_data(adjusted_df, 
                             on_columns= ['cohort', 'gender'],
                             step='M')
                                                       

,cohort,gender,time_since_attribution,revenue,monthly_subs,annual_subs,current_date,cumulative_revenue,time_since_attribution_time0,revenue_time0,monthly_subs_time0,annual_subs_time0,current_date_time0,cumulative_revenue_time0,time_purchase
0,2017-01-01,female,0,1566875,295,1275,2017-01-01,1566875,0,1566875,295,1275,2017-01-01,1566875,2017-01-01
1,2017-01-01,female,1,36750,294,1271,2017-02-01,1603625,0,1566875,295,1275,2017-01-01,1566875,2017-02-01
2,2017-01-01,female,2,36125,289,1261,2017-03-01,1639750,0,1566875,295,1275,2017-01-01,1566875,2017-03-01
3,2017-01-01,female,3,35750,286,1251,2017-04-01,1675500,0,1566875,295,1275,2017-01-01,1566875,2017-04-01
4,2017-01-01,female,4,35000,280,1246,2017-05-01,1710500,0,1566875,295,1275,2017-01-01,1566875,2017-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555,2021-12-01,male,8,278000,2224,2198,2022-08-01,5150125,0,2926125,2241,2205,2021-12-01,2926125,2022-08-01
1556,2021-12-01,male,9,278000,2224,2198,2022-09-01,5428125,0,2926125,2241,2205,2021-12-01,2926125,2022-09-01
1557,2021-12-01,male,10,278000,2224,2198,2022-10-01,5706125,0,2926125,2241,2205,2021-12-01,2926125,2022-10-01
1558,2021-12-01,male,11,278000,2224,2198,2022-11-01,5984125,0,2926125,2241,2205,2021-12-01,2926125,2022-11-01


In [22]:
adjusted_df['gf_actual'] = adjusted_df['cumulative_revenue'] / adjusted_df['revenue_time0']
adjusted_df

,cohort,gender,time_since_attribution,revenue,monthly_subs,annual_subs,current_date,cumulative_revenue,time_since_attribution_time0,revenue_time0,monthly_subs_time0,annual_subs_time0,current_date_time0,cumulative_revenue_time0,time_purchase,gf_actual
0,2017-01-01,female,0,1566875,295,1275,2017-01-01,1566875,0,1566875,295,1275,2017-01-01,1566875,2017-01-01,1.000000
1,2017-01-01,female,1,36750,294,1271,2017-02-01,1603625,0,1566875,295,1275,2017-01-01,1566875,2017-02-01,1.023454
2,2017-01-01,female,2,36125,289,1261,2017-03-01,1639750,0,1566875,295,1275,2017-01-01,1566875,2017-03-01,1.046510
3,2017-01-01,female,3,35750,286,1251,2017-04-01,1675500,0,1566875,295,1275,2017-01-01,1566875,2017-04-01,1.069326
4,2017-01-01,female,4,35000,280,1246,2017-05-01,1710500,0,1566875,295,1275,2017-01-01,1566875,2017-05-01,1.091663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555,2021-12-01,male,8,278000,2224,2198,2022-08-01,5150125,0,2926125,2241,2205,2021-12-01,2926125,2022-08-01,1.760050
1556,2021-12-01,male,9,278000,2224,2198,2022-09-01,5428125,0,2926125,2241,2205,2021-12-01,2926125,2022-09-01,1.855056
1557,2021-12-01,male,10,278000,2224,2198,2022-10-01,5706125,0,2926125,2241,2205,2021-12-01,2926125,2022-10-01,1.950062
1558,2021-12-01,male,11,278000,2224,2198,2022-11-01,5984125,0,2926125,2241,2205,2021-12-01,2926125,2022-11-01,2.045068


In [23]:
# save to Data folder
adjusted_df.to_csv('../Data/adjusted_data.csv', index=False)